# Our first LCA

We want to do our first LCA. For starters, we will analyze the ecoinvent activity "market for electricity, medium voltage" in Germany. Make sure you have created a project and imported the ecoinvent database.

In [1]:
# imports
import brightway2 as bw

We forgot the name of the project where we improted ecoinvent. Let's see a list of all projects to remember.

In [2]:
bw.projects

Brightway2 projects manager with 4 objects:
	default
	ecoinvent-import
	ei_3.5
	new_project
Use `projects.report()` to get a report on all projects.

Ah yeah, it was called new_project! Let's open it.

In [4]:
bw.projects.set_current("new_project")

What was the name that we gave to the database again? Let's check all database names in the current project.

In [5]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei35

Oh yeah, ei35! Okay. Let's grab the activity.

In [8]:
results = bw.Database("ei35").search("market for electricity, medium voltage")
results

['market for electricity, medium voltage' (kilowatt hour, BR, None),
 'market for electricity, medium voltage' (kilowatt hour, US-HICC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-TRE, None),
 'market for electricity, medium voltage' (kilowatt hour, US-ASCC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-FRCC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-NPCC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-RFC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-WECC, None),
 'market for electricity, medium voltage' (kilowatt hour, US-MRO, None),
 'market for electricity, medium voltage' (kilowatt hour, US-SERC, None),
 'market for electricity, medium voltage' (kilowatt hour, CN-SGCC, None),
 'market for electricity, medium voltage' (kilowatt hour, CN-CSG, None),
 'market for electricity, medium voltage' (kilowatt hour, CA-AB, None),
 'market for electricity, medium voltage' (kilowa

Oops, forgot to specifiy the location. Let's try again:

In [9]:
results = bw.Database("ei35").search("market for electricity, medium voltage", filter={"location":"DE"})
results

Excluding 644 filtered results


['market for electricity, medium voltage' (kilowatt hour, DE, None),
 'electricity, from municipal waste incineration to generic market for electricity, medium voltage' (kilowatt hour, DE, None),
 'market for electricity, high voltage' (kilowatt hour, DE, None),
 'market for electricity, low voltage' (kilowatt hour, DE, None),
 'electricity, high voltage, production mix' (kilowatt hour, DE, None)]

Yep, the first entry is the one. Let's save it.

In [10]:
act = results[0]

Something important to notice: Brightway search will return results according to an internal prioritization. Do **not** expect the order of the returned results to be always the same. If you want to be sure to get the right activity, use its unique "code", like so:

In [35]:
code = act["code"]
db = act["database"]
query = (db,code)
query

('ei35', '027a5a09232e5a810a31d5cf92cc29b0')

In [36]:
bw.get_activity(query)

'market for electricity, medium voltage' (kilowatt hour, DE, None)

The ecoinvent database in brightway is built such that every activity has exactly **one** main product. When we can figure out what the main product of an activity is by using the function `production()`. This function will return a handler to the database which is not very useful here. To get more human-readable output, we convert it to a list and pick the first entry.

In [59]:
list(act.production())[0]

Exchange: 1.0 kilowatt hour 'market for electricity, medium voltage' (kilowatt hour, DE, None) to 'market for electricity, medium voltage' (kilowatt hour, DE, None)>

For the LCA, brightway expects a functional unit. Normally, you would expect something like "1 kWh of electricity produced". In brightway, the functional unit instead is "1 market for electricity, medium voltage". Because every activity is associated with exactly one main product, this definition is still univocal.

In [61]:
functional_unit = {act:1}

To conduct an LCA we do not just need the functional unit but also a method to score the activity. Let's see which methods brightway has to offer.

In [37]:
bw.methods

Methods dictionary with 850 objects, including:
	('CML 2001 (obsolete)', 'acidification potential', 'average European')
	('CML 2001 (obsolete)', 'acidification potential', 'generic')
	('CML 2001 (obsolete)', 'climate change', 'GWP 100a')
	('CML 2001 (obsolete)', 'climate change', 'GWP 20a')
	('CML 2001 (obsolete)', 'climate change', 'GWP 500a')
	('CML 2001 (obsolete)', 'climate change', 'lower limit of net GWP')
	('CML 2001 (obsolete)', 'climate change', 'upper limit of net GWP')
	('CML 2001 (obsolete)', 'eutrophication potential', 'average European')
	('CML 2001 (obsolete)', 'eutrophication potential', 'generic')
	('CML 2001 (obsolete)', 'freshwater aquatic ecotoxicity', 'FAETP 100a')
Use `list(this object)` to get the complete list.

850 methods! That's plenty. Unfortunately, there is no search function for them. We need to use python list comprehension to get the ones we want. Let's go for ILCD 2.0 midpoint.

In [44]:
[m for m in bw.methods if "ILCD 2.0 2018 midpoint" in str(m)]

[('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'freshwater ecotoxicity'),
 ('ILCD 2.0 2018 midpoint', 'human health', 'non-carcinogenic effects'),
 ('ILCD 2.0 2018 midpoint', 'human health', 'carcinogenic effects'),
 ('ILCD 2.0 2018 midpoint', 'resources', 'minerals and metals'),
 ('ILCD 2.0 2018 midpoint no LT', 'resources', 'minerals and metals'),
 ('ILCD 2.0 2018 midpoint', 'climate change', 'climate change biogenic'),
 ('ILCD 2.0 2018 midpoint', 'climate change', 'climate change fossil'),
 ('ILCD 2.0 2018 midpoint',
  'climate change',
  'climate change land use and land use change'),
 ('ILCD 2.0 2018 midpoint', 'climate change', 'climate change total'),
 ('ILCD 2.0 2018 midpoint',
  'ecosystem quality',
  'freshwater and terrestrial acidification'),
 ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'freshwater eutrophication'),
 ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'marine eutrophication'),
 ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'terrestrial eutrophication'),

There are two versions: long-term and no long-term. We choose the former.

In [45]:
ilcd = [m for m in bw.methods if "ILCD 2.0 2018 midpoint" in str(m) and "LT" not in str(m)]
ilcd

[('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'freshwater ecotoxicity'),
 ('ILCD 2.0 2018 midpoint', 'human health', 'non-carcinogenic effects'),
 ('ILCD 2.0 2018 midpoint', 'human health', 'carcinogenic effects'),
 ('ILCD 2.0 2018 midpoint', 'resources', 'minerals and metals'),
 ('ILCD 2.0 2018 midpoint', 'climate change', 'climate change biogenic'),
 ('ILCD 2.0 2018 midpoint', 'climate change', 'climate change fossil'),
 ('ILCD 2.0 2018 midpoint',
  'climate change',
  'climate change land use and land use change'),
 ('ILCD 2.0 2018 midpoint', 'climate change', 'climate change total'),
 ('ILCD 2.0 2018 midpoint',
  'ecosystem quality',
  'freshwater and terrestrial acidification'),
 ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'freshwater eutrophication'),
 ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'marine eutrophication'),
 ('ILCD 2.0 2018 midpoint', 'ecosystem quality', 'terrestrial eutrophication'),
 ('ILCD 2.0 2018 midpoint', 'human health', 'ionising radiation'),
 ('

I recommend saving this snippet in a separate function for easy reuse!

Now that we have the functional unit and the methods, we put both together and store it as a new calculation.

In [68]:
setup = {'inv': [functional_unit], 'ia': ilcd}
setup_name = "electicity_DE"
bw.calculation_setups[setup_name] = setup

To run the calculation, we create a MultiLCA object.

In [71]:
mlca = bw.MultiLCA(setup_name)
mlca.results

array([[1.53498136e-01, 5.31023658e-08, 4.53285278e-09, 3.45861718e-07,
        3.93489673e-03, 6.37650317e-01, 1.30762370e-03, 6.42892837e-01,
        2.25379800e-03, 9.42844699e-04, 5.19175559e-04, 6.66115727e-03,
        1.36384537e-01, 3.87192907e-08, 8.64925670e-04, 1.04041589e-08,
        1.16587946e-01, 1.11444577e+01, 4.46247451e+00]])

The numbers alone are not very descriptive so let's add the method names.

In [73]:
{k:v for k,v in zip(ilcd, mlca.results[0])}

{('ILCD 2.0 2018 midpoint',
  'ecosystem quality',
  'freshwater ecotoxicity'): 0.15349813647937102,
 ('ILCD 2.0 2018 midpoint',
  'human health',
  'non-carcinogenic effects'): 5.3102365829456645e-08,
 ('ILCD 2.0 2018 midpoint',
  'human health',
  'carcinogenic effects'): 4.532852778325799e-09,
 ('ILCD 2.0 2018 midpoint',
  'resources',
  'minerals and metals'): 3.458617184659298e-07,
 ('ILCD 2.0 2018 midpoint',
  'climate change',
  'climate change biogenic'): 0.003934896733133886,
 ('ILCD 2.0 2018 midpoint',
  'climate change',
  'climate change fossil'): 0.6376503167066483,
 ('ILCD 2.0 2018 midpoint',
  'climate change',
  'climate change land use and land use change'): 0.0013076236953589364,
 ('ILCD 2.0 2018 midpoint',
  'climate change',
  'climate change total'): 0.6428928371351411,
 ('ILCD 2.0 2018 midpoint',
  'ecosystem quality',
  'freshwater and terrestrial acidification'): 0.0022537979965747026,
 ('ILCD 2.0 2018 midpoint',
  'ecosystem quality',
  'freshwater eutrophicati

There we have it! A life cycle impact assessment of electricity production in Germany. To do: Visualization.